In [ ]:
###Neural Net####
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from IPython.display import Image  
import pydotplus
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn import metrics

clfNN = MLPClassifier(activation = 'relu', 
                      solver='lbfgs', alpha=1e-5,
                      hidden_layer_sizes=(10, 2), 
                      random_state=1, 
                      max_iter=5000)

cv = StratifiedKFold(n_splits=20, random_state=0, shuffle=True)

from sklearn.preprocessing import StandardScaler  # doctest: +SKIP
scaler = StandardScaler()  # doctest: +SKIP

totScore=0
totSensitivity = 0
totSpecificity = 0
totAUC = 0
i=0

#print(dfSelectedX.head())
for train_index,test_index in cv.split(dfSelectedX,dfSelectedY):
    x_train, x_test = dfSelectedX.iloc[train_index], dfSelectedX.iloc[test_index]
    y_train, y_test = dfSelectedY.iloc[train_index], dfSelectedY.iloc[test_index]
    
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    
    clfNN.fit(x_train, y_train.values.ravel())
    predictions = clfNN.predict(x_test)

    predY = pd.DataFrame(predictions,index= y_test.index, columns=['Predicted_type'])
    #print(predY)

    finalPred = pd.concat([y_test,predY],axis=1)
    #print(finalPred)
    #print(multilabel_confusion_matrix(y_test, predictions))
    
    #print("\n-----------------------------")
    tn,fp,fn,tp = confusion_matrix(y_test, predictions).ravel()
    #print(" tn\tfp\tfn\t tp\n",tn,"\t",fp,"\t",fn,"\t",tp)
    score = clfNN.score(x_test, y_test)
    #print("\naccuracy:",score)
    
    #print("-----------------------------\n")
    score = clfNN.score(x_test, y_test)
    sensitivity = tp*100/(tp+fn)
    specificity = tn*100/(tn+fp)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, predictions)
    auc = metrics.auc(fpr, tpr)
    #print(score)
    
    totScore = totScore + score
    totSensitivity = totSensitivity + sensitivity
    totSpecificity = totSpecificity + specificity
    totAUC = totAUC + auc
    i=i+1
    
print("average accuracy:", totScore/i)
print("average sensitivity:", totSensitivity/i)
print("average specificity:", totSpecificity/i)
print("average AUC:", totAUC/i)